# **Import Libraries & Data Loading**

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import re # if u want to learn regex [https://regex101.com/]
import string
import random # Random number generators - Library for generating random numbers, selecting random elements, shuffling sequences, etc.
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
from plotly import graph_objs as go
import plotly.express as px
import plotly.figure_factory as ff
from collections import Counter

from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

pd.set_option("display.max_columns", None)
# pd.set_option("display.max_row", None)

In [2]:
!pip install sastrawi nlp-id

In [3]:
import nltk # Natural Language Toolkit - Library for natural language processing (NLP) tasks such as tokenization, stemming, tagging, parsing, and more.
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from tqdm import tqdm # A Fast, Extensible Progress Bar - Library for creating progress bars to monitor the progress of iterations or tasks.
from nltk.stem import WordNetLemmatizer, PorterStemmer

from nlp_id.lemmatizer import Lemmatizer
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from tensorflow.keras.preprocessing.text import Tokenizer

2024-06-16 06:38:16.787672: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-16 06:38:16.787781: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-16 06:38:16.789774: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
# Hapus @, hapus link, reply message nya di ilangin juga, non ascii ilangin atau replace. yang typo typo bisa maping manual, atau cari cara nya () spell checker namanya.
# Vectorize terus kemodel kalau udah slesai preprocessing (kalau mau manual)

# TF-IDF, Word2Vec, dll

# kalau bert itu mau kata kata yang utuh, jadi jangan terlalu aneh aneh untuk processing. jadi intinya processing sederhana

# Coba Finetune bert model, biar bisa sesuai dengan kasus yang sedang dihadapi

# terus ada lagi yang namanya sentence transformers, kalau bert itu kata kata masukin model, terus dapet labelnya apa. tpi kalau dsi sentence transformers itu ngambil vector di layer tertentu. misalkan ada kata kata, terus kitam auskin ke model, nah dia ngasih dalam bentuk vektor bukan labelnya, nah enaknya hasil vektor ini enaknya adalah, bisa digabungin oleh jumlah label lain, misalkan catboost dll nya.

# coba pakai indo bert tweet, bukan indobert biasa (nama model nya) atau coba dibandingkan aja. kalau mau pake bert tweet, jangan hapus hastag nya jangan diolah aneh aneh.

# Saran coba pakai catboost ajah. catboostclassifier(Text_feature = ["kolommu"]) terus tinggal di fit aja.

# coba liat wordcloud untuk bantuan membedakan setiapa labelnya, jadi Text yang paling gede(sering muncul) apakah ada perbedaan diantara label nya.

# bisa juga buat featrue extraction, tmabah kolom, is_debat, is_capres, nah jadi kayak 1 0 1 0 gitu.

# bisa juga pake post tagging, jadi nnti ada kolom baru, jumlah subject nya, jumlah adjectivenya dll lah.

# pertama coba baseline dlu, baru coba yang aneh aneh nya processing nya.

# ubah label yang inconsistent, coba coba aja. boleh kok processing data di label nya juga.

# catboost classifier for class imbalanced, scale_pos_weight. jadi menanggulangi class imbalance, coba coba aja sih.

In [5]:
def check_duplicates(dataframe):
    print("Duplicate Values (Top 10):")
    duplicate_values = dataframe[dataframe.duplicated()]
    print(f"Number of Duplicate Rows: {duplicate_values.shape[0]}")
    display(duplicate_values.head(10))

def missing_data(data):
    total = data.isnull().sum().sort_values(ascending = False)
    percent = (data.isnull().sum()/data.isnull().count()*100).sort_values(ascending = False)
    return pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])

def basic_data_info(dataframe):
    print("Data Preview:")
    print("---------------------------")
    display(dataframe.head())

    print("\nGeneral Info:")
    print("---------------------------")
    print(dataframe.info())

    print("\nDescriptive Statistics:")
    print("---------------------------")
    display(dataframe.describe().T)

# **Load Data**

In [6]:
test = pd.read_csv('/kaggle/input/satriadata/dataset_unlabeled_penyisihan_bdc_2024.csv', sep=";")
submissions = pd.read_csv('/kaggle/input/satriadata/template_jawaban_penyisihan_bdc_2024.csv', sep=";")
lexicon_corpus = pd.read_excel('/kaggle/input/satriadata/colloquial-indonesian-lexicon.xlsx', sheet_name="Clean-Colloquial-Lexicon")
stopword_corpus = pd.read_excel('/kaggle/input/satriadata/Stopword byTimTam.xlsx')

# **Simple Explore the Data**

In [7]:
check_duplicates(test)
# drop duplicate entries considering all columns
test = test.drop_duplicates()

Duplicate Values (Top 10):
Number of Duplicate Rows: 0


,IDText,Text


In [8]:
missing_data(test)

,Total,Percent
IDText,0,0.0
Text,0,0.0


In [9]:
basic_data_info(test)

Data Preview:
---------------------------


,IDText,Text
0,TXT0001,Lu mau org2 pro-demokrasi di negara ini bisa p...
1,TXT0002,Prabowo ditanya soal hutang luar negeri dia me...
2,TXT0003,kiki_daliyo Ganjar Pranowo itulah beliau soso...
3,TXT0004,@kumparan Prabowo Gibran yang bisa melakukan i...
4,TXT0005,@sniperruben45 @uda_zulhendra @ainunnajib Lah ...



General Info:
---------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   IDText  1000 non-null   object
 1   Text    1000 non-null   object
dtypes: object(2)
memory usage: 15.8+ KB
None

Descriptive Statistics:
---------------------------


,count,unique,top,freq
IDText,1000,1000,TXT0001,1
Text,1000,1000,Lu mau org2 pro-demokrasi di negara ini bisa p...,1


# **Feature Engineering**

> > ### ***Preprocessing Data***
---

### Text Processing **_[clean_Text_1]_**

1. Mengubah semua karakter dalam teks menjadi **huruf kecil.**
2. Menghapus **'RT'**: Menggunakan pola \bRT\b untuk mencocokkan kata 'RT' sebagai kata tunggal.
3. Menghapus **link**: Menggunakan pola http\S+|www\S+ untuk menghapus link.
4. Menghapus pola **'[RE ...]'**: Menggunakan pola \[RE [^]]+\] untuk menghapus teks yang diawali dengan [RE dan diakhiri dengan ].
5. Menghapus pola **'@ ... ='**: Menggunakan pola @\S+= untuk menghapus teks yang diawali dengan @ dan diakhiri dengan =.
6. Menghapus **karakter non-ASCII (termasuk emoji)**: Menggunakan pola [^\x00-\x7F]+ untuk menghapus karakter non-ASCII.
7. Menghapus **spasi berlebih**: Menggunakan pola \s+ untuk mengganti spasi berlebih dengan satu spasi.
8. Menghapus **Tanda Baca**: Menggunakan translate string punctuation
9. Menghapus **Angka**: Menggunakan regex \b\d+\b untuk menghapus angka.
10. menghapus **single char**: Menggunakan regex "\b[a-zA-Z]\b" untuk menghapus karakter tunggal.
11. mengganti kata **slang** menjadi kata **formal**: mengganti kata slang dengan kata formal dalam teks menggunakan ekspresi reguler. Setiap pasangan kata slang dan formal dari slang_formal_pairs akan diaplikasikan ke teks.

In [10]:
# for i in (lexicon_corpus[lexicon_corpus.isnull().any(axis=1)]).slang:
#   print(i)

In [11]:
# drop nilai nan di dalam corpus lexicon, yang kata formalnya tidak ada
lexicon_corpus = lexicon_corpus.dropna(subset=['formal'])

def replace_slang(Text, slang_formal_pairs):
    for slang, formal in slang_formal_pairs:
        Text = re.sub(r'\b{}\b'.format(re.escape(slang)), formal, Text)
    return Text

slang_formal_pairs = list(zip(lexicon_corpus['slang'], lexicon_corpus['formal']))

In [12]:
# Fungsi untuk membersihkan teks
def clean_Text(Text, slang_formal_pairs = slang_formal_pairs):
    # Lowercasing
    Text = Text.lower()
    # Menghapus 'RT'
    Text = re.sub(r'\brt\b', '', Text)
    # Menghapus link (link tweet)
    Text = re.sub(r'http\S+|www\S+', '', Text)
    # Menghapus pola '[RE ...]'
    Text = re.sub(r'\[re [^]]+\]', '', Text)
    # Menghapus pola '@ ... ='
    Text = re.sub(r'@\S+=', '', Text)
    # Menghapus karakter non-ASCII (termasuk emoji)
    Text = re.sub(r'[^\x00-\x7F]+', '', Text)
    # Menghapus spasi berlebih
    Text = re.sub(r'\s+', ' ', Text).strip()
# # Menghapus tanda baca dari kalimat
# Text = Text.translate(str.maketrans("", "", string.punctuation))
    # Menghapus tanda baca kecuali '#'
    Text = re.sub(r'[{}]'.format(re.escape(string.punctuation.replace('#', ''))), '', Text)
    # Menghapus angka
    Text = re.sub(r'\b\d+\b', '', Text)
    # Menghapus karakter tunggal (single character)
    Text = re.sub(r"\b[a-zA-Z]\b", " ", Text)
    # Mengganti kata-kata dengan kata yang sesuai dari lexicon_corpus
    Text = replace_slang(Text, slang_formal_pairs)

    return Text

# Menerapkan fungsi ke kolom Text
test['clean_Text_1'] = test['Text'].apply(clean_Text)

In [13]:
for tweet in test.sample(5, random_state= 2035)["Text"]:
  print(tweet)

@cuitcuap_ sepakat nih, MASBowoGBran PASTInya peringatkan kepada semua wisudawan dan masyarakat agar menghindar diajak percaya pada ajaran yg keras.

#BanyakYgBalikKe02
KarenaHATI BAIKPakBowo
@prabowo @prabowogibran02
Ganjar-Mahfud adalah solusi yang tepat untuk mewujudkan Indonesia yang lebih merata dalam pertumbuhan ekonomi. @Arunaurgf___ Happy New Year 
#GanjarMahfud2024
#GanjarMahfudM3nang
#L3bihbaik https://t.co/aShK4JchZ2
@ganjarpranowo @aniesbaswedan @mohmahfudmd @prabowo Ganjar Mahfud taat pada hukum dan konstitusi, Ganjar Mahfud adalah peluru yg tak terkendali dalam pemberantasan korupsi di Indonesia.
#GanjarMahfud2024 https://t.co/VfVzXFFuPD
Ganjar di sini paling keren dan memahami sih sob. Kongkrit dan terukur.

Pertanyaan 6. Globalisasi. Strategi promosi budaya populer nusantara untuk pertumbuhan ekonomi

ABW
Alokasi SD yang cukup di kebudayaan. Dipandang sbg investasi. Kuliner, Seni, Film tumbuh dan dibawa global
Bersama Ganjar Pranowo kita wujudkan kesejahteraan yang adil

In [14]:
for tweet in test.sample(5, random_state= 2035)["clean_Text_1"]:
  print(tweet)

cuitcuap sepakat nih prabowo gibran pastinya peringatkan kepada semua wisudawan dan masyarakat agar menghindar diajak percaya pada ajaran yang keras #banyakygbalikke02 karena hati baik prabowo prabowo prabowogibran02
ganjar mahfud adalah solusi yang tepat untuk mewujudkan indonesia yang lebih merata dalam pertumbuhan ekonomi arunaurgf bahagia baru tahun #ganjar mahfud #ganjarmahfudm3nang #l3bihbaik
ganjarpranowo aniesbaswedan moh mahfud md prabowo ganjar mahfud taat pada hukum dan konstitusi ganjar mahfud adalah peluru yang tak terkendali dalam pemberantasan korupsi di indonesia #ganjar mahfud
ganjar di sini paling keren dan memahami sih sob kongkrit dan terukur pertanyaan  globalisasi strategi promosi budaya populer nusantara untuk pertumbuhan ekonomi abw alokasi sampai dengan yang cukup di kebudayaan dipandang sebagai investasi kuliner seni film tumbuh dan dibawa global
bersama ganjar pranowo kita wujudkan kesejahteraan yang adil dan merata ganjar mahfud bandung bandung ganjar mahfud

### Text Processing **_[clean_Text_2]_**

1. Menggunakan **clean_Text_1** ditambahkan dengan ***penghapusan untuk hastag nya***

In [15]:
def remove_hashtag(Text):
  Text = re.sub(r'#\w+', '', Text)
  # Menghapus spasi berlebih
  Text = re.sub(r'\s+', ' ', Text).strip()

  return Text

In [16]:
# Menerapkan fungsi ke kolom Text
test['clean_Text_2'] = test['clean_Text_1'].apply(remove_hashtag)

### Text Processing **_[clean_Text_3]_**

1. Menggunakan clean_Text_2 ditambahkan dengan stop_word, lemmatization, Stemming (NLTK, nlp-id, Sastrawi)

In [17]:
nltk.download(['stopwords', 'punkt', 'wordnet'])

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [18]:
# Initialize stopwords, lemmatizer, and stemmer

stop_words = set(stopwords.words('indonesian')) # NLTK
lemmatizer = Lemmatizer() # nlp-id

factory = StemmerFactory() # Sastrawi for stemming (Indonesian) nltk just have English Stemmer
stemmer_sastrawi = factory.create_stemmer()

# Add custom stopwords
custom_stopwords = {
    'yg', 'dgn', 'dg', 'd', 'kalo', 'ya', 'jd', 'jgn', 'sdh', 'aja', 'tuh', 'nih',
    'sih', 'si', 'deh', 'loh', 'bakal', 'gak', 'ga', 'gitu', 'sampe', 'amp', 'rt',
    'dm', 'url', 'user', 'https', 'http', 'www', 'com', "..", "..."
}
stop_words.update(custom_stopwords)

In [19]:
# Define the preprocessing function
def preprocess_Text(Text):

    # Tokenize the Text into words
    words = Text.split()

    # Remove stopwords and apply stemming and lemmatization
    processed_words = []
    for word in words:
        if word.lower() not in stop_words: # NLTK Stopword
            stemmed_word = stemmer_sastrawi.stem(word) # Sastrawi Stemming
            lemmatized_word = lemmatizer.lemmatize(stemmed_word) # npl-id
            processed_words.append(lemmatized_word)

    # Join the processed words back into a single string
    return " ".join(processed_words)

In [20]:
# Apply the preprocessing function to the 'Text' column
test['clean_Text_3'] = test['clean_Text_2'].apply(preprocess_Text)

### Text Processing **_[clean_Text_4]_**

1. Menggunakan **Text awal**, namun untuk **processing tex**t nya akan dibantu oleh library **tweet-preprocessor** untuk processing Text nya

In [21]:
pip install tweet-preprocessor

Note: you may need to restart the kernel to use updated packages.


In [22]:

import preprocessor as p

In [23]:
def preprocess_tweet(row):
    Text = row['Text']
    # Lowercasing
    Text = Text.lower()
    # Menghapus pola '@ ... ='
    Text = re.sub(r'@\S+=', '', Text)
    Text = p.clean(Text)
    return Text

In [24]:
test['clean_Text_4'] = test.apply(preprocess_tweet, axis=1)

In [25]:
test = test.reset_index(drop = True)

In [26]:
# Define the labels
labels = [
    "Politik",
    "Sosial Budaya",
    "Ideologi",
    "Pertahanan dan Keamanan",
    "Ekonomi",
    "Sumber Daya Alam",
    "Demografi",
    "Geografi"
]

### Text Processing **_[clean_Text_5]_**

Try to **handling** the **english and indonesian word**, here we should get the only **Indo word**, or **just get both** and **translate english into indonesian**

In [27]:
!pip install langdetect

In [28]:
from langdetect import detect, detect_langs

# Download stopwords
nltk.download('stopwords')
indonesian_stopwords = set(stopwords.words('indonesian'))
english_stopwords = set(stopwords.words('english'))

# Custom stopwords for Twitter
custom_stopwords = {
    'yg', 'dgn', 'dg', 'd', 'kalo', 'ya', 'jd', 'jgn', 'sdh', 'aja', 'tuh', 'nih',
    'sih', 'si', 'deh', 'loh', 'bakal', 'gak', 'ga', 'gitu', 'sampe', 'amp', 'rt',
    'dm', 'url', 'user', 'https', 'http', 'www', 'com'
}

indonesian_stopwords.update(custom_stopwords)

# Define the function to preprocess and classify words
def preprocess_and_classify(Text):
    # Remove mentions, hashtags, and URLs
    Text = re.sub(r'@\w+|#\w+|http\S+', '', Text)

    # Tokenize the Text into words
    words = Text.split()

    english_words = []
    other_words = []

    for word in words:
        # Remove stopwords
        if word.lower() in indonesian_stopwords or word.lower() in english_stopwords:
            continue

        # Classify the word
        try:
            lang = detect(word)
            if lang == 'en':
                english_words.append(word)
            else:
                other_words.append(word)
        except:
            other_words.append(word)

    return {
        'english': english_words,
        'other': other_words
    }

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# **Saving and load Data**

In [29]:
# # # Save the DataFrame to a CSV file
test.to_csv("/kaggle/working/Processing-Data-clean-Text-4.csv", index=False)

In [30]:
test = pd.read_csv("/kaggle/working/Processing-Data-clean-Text-4.csv")

# **Preprocessing with Stopword and Corpus Lexicon Tambahan**

## Text Processing **_[clean_Text_5]_**

1. Menggunakan/menyempurnakan Text **"Clean_Text_3"** namun terdapat **tambahan processing** yaitu dengan menambhkan **stopword dan lexicon corpus** from **TimTam** yang dibuat **secara manual check by each sentences**

### Clean Lexicon Corpus

In [31]:
# drop nilai nan di dalam corpus lexicon, yang kata formalnya tidak ada
lexicon_corpus = lexicon_corpus.dropna(subset=['formal'])

def replace_slang(Text, slang_formal_pairs):
    for slang, formal in slang_formal_pairs:
        Text = re.sub(r'\b{}\b'.format(re.escape(slang)), formal, Text)
    return Text

slang_formal_pairs = list(zip(lexicon_corpus['slang'], lexicon_corpus['formal']))

In [32]:
# Fungsi untuk membersihkan teks
def clean_Text(Text, slang_formal_pairs = slang_formal_pairs):
    # Lowercasing
    Text = Text.lower()
    # Mengganti kata-kata dengan kata yang sesuai dari lexicon_corpus
    Text = replace_slang(Text, slang_formal_pairs)

    return Text

In [33]:
# Menerapkan fungsi ke kolom Text
test['clean_Text_5'] = test['clean_Text_3'].apply(clean_Text)

### Clean Stopword

In [34]:
# # Save the DataFrame to a CSV file
test.to_csv("/kaggle/working/Processing-Data-clean-Text-5.csv", index=False)

test = pd.read_csv("/kaggle/working/Processing-Data-clean-Text-5.csv")

In [35]:
# Download stopwords and wordnet
nltk.download(['stopwords', 'wordnet'])

# Initialize Sastrawi stemmer
factory = StemmerFactory()
stemmer_sastrawi = factory.create_stemmer()

# Define stop words
stop_words = set(stopwords.words('indonesian'))
stop_words.update(stopword_corpus["Stopword"])

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [36]:
# for i in test['clean_Text_5']:
#   print(i)

In [37]:
# Define the preprocessing function
def preprocess_Text(Text):
    if not isinstance(Text, str):
        return ""

    # Tokenize the Text into words
    words = Text.split()

    # Remove stopwords and apply stemming
    processed_words = []
    for word in words:
        if word.lower() not in stop_words:  # NLTK Stopword
            stemmed_word = stemmer_sastrawi.stem(word)  # Sastrawi Stemming
            processed_words.append(stemmed_word)

    # Join the processed words back into a single string
    return " ".join(processed_words)

# Apply the preprocessing function to the 'clean_Text_5' column
test['clean_Text_5'] = test['clean_Text_5'].astype(str).apply(preprocess_Text)

# # Apply the preprocessing function to the 'clean_Text_5' column
# test['clean_Text_5'] = test['clean_Text_5'].apply(preprocess_Text)

In [38]:
for i in test["clean_Text_5"]:
  print(i)

orang prodemokrasi negara lawan oligarki indonesia pilih amin pilih pks pilih partai buruh pilih prabowo partai usung pilih pilih partai demokrasi indonesia juang pilih partai buruh
prabowo hutang negeri hutang negeri indonesia hitung rendah khawatir intervensi asing akibat hutang krisis moneter orde ban ganjar hutang negeri infrastruktur bahaya omong baju ganjar matching tema debat anies normal hutang negeri aman gdp hutang pakai menteri tahan beli alat utama sistem senjata bekas kesiasiaan prabowo merespon alat utama sistem senjata bekas beli uang
kikidaliyo ganjar pranowo beliau sosok mengagunhkan nilai nilai pancasila sadar hebat mahfud bahagia
kumpar prabowo gibran sejahtera rakyat
sniperruben45 udazulhendra ainunnajib sambung junjung aomkmkmkmk omong etika hina anies omong nyenyek anies sopan akak brutal pasang wajah ejek asbun hutang nyuruh ajar ekonomi
nelayan jebak kredit macet ganjar pranowo mahfud buka pintu bebas
prabowo anti bebas pers
kontrak buka ganjar pranowo cermin pe

In [39]:
for i in test["clean_Text_3"].head(10):
  print(i)

orang prodemokrasi negara sempat lawan oligarki indonesia pilih amin pilih pks pilih partai buruh pilih prabowo partai usung pilih gama pilih partai demokrasi indonesia juang pilih partai buruh
prabowo hutang negeri hutang negeri indonesia hitung rendah khawatir intervensi asing akibat hutang krisis moneter orde ban ganjar hutang negeri infrastruktur bahaya omong baju ganjar matching tema debat anies normal hutang negeri aman gdp hutang pakai menteri tahan beli alat utama sistem senjata bekas kesiasiaan prabowo merespon alat utama sistem senjata bekas beli uang keluar
kikidaliyo ganjar pranowo beliau sosok mengagunhkan nilai nilai pancasila sadar hebat mahfud bahagia
kumpar prabowo gibran sejahtera rakyat
sniperruben45 udazulhendra ainunnajib sambung junjung aomkmkmkmk omong etika katai anies omong nyenyek anies sopan lho akak brutal pasang wajah ejek asbun hutang nyuruh ajar ekonomi
nelayan jebak kredit macet ganjar pranowo mahfud md buka pintu bebas
prabowo anti bebas pers
kontrak bu

In [40]:
# # Save the DataFrame to a CSV file
test.to_csv("/kaggle/working/Processing-Data-clean-Text-5.csv", index=False)

# test = pd.read_csv(cleanDataPath + "/Processing-Data-clean-Text-5.csv")